In [7]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pickle

In [27]:
data = pd.read_csv('TCGA-PANCAN-HiSeq-801x20531/data.csv', index_col = 0)
#labels = pd.read_csv('TCGA-PANCAN-HiSeq-801x20531/labels.csv')
eigenvalues = pickle.load( open( "eigenvalues_first_800.pkl", "rb" ) )
eigenvectors = pickle.load( open( "eigenvectors_first_800.pkl", "rb" ) )

In [28]:
mean_arr = np.mean(data, 0)
var_arr = np.var(data, 0)
df = data.drop(data.columns[np.argwhere(np.array(var_arr == 0)).flatten()], axis = 1)
mean_arr = np.mean(df, 0)
var_arr = np.var(df, 0)
min_var = 2
df.drop(df.columns[np.argwhere(np.array(var_arr < min_var)).flatten()], axis = 1, inplace = True)
mean_arr = np.mean(df, 0)
var_arr = np.var(df, 0)
df = (df - mean_arr)/(var_arr ** (1/2))
df = np.real(np.array(df@eigenvectors))

In [30]:
n = df.shape[0]


801
